In [6]:
## model checkpoints using notebook_login to connect HuggingFace
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import torch
from torch.utils.data import DataLoader, Dataset

In [8]:
class KWS_dataset(Dataset):
    def __init__(self, input_data, output_data):
        self.input_data = input_data
        self.output_data = output_data
        
    def __len__(self):
        return len(self.input_data)
    
    def __getitem__(self, index):
        keyword = self.output_data[index]
        audio_features = self.input_data[index]
        return audio_features, keyword

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
train_loader = torch.load('/content/drive/MyDrive/Colab Notebooks/data/en_splits_10.trainloader')
dev_loader = torch.load('/content/drive/MyDrive/Colab Notebooks/data/en_splits_10.devloader')
test_loader = torch.load('/content/drive/MyDrive/Colab Notebooks/data/en_splits_10.testloader')

In [13]:
print(len(train_loader.dataset))
print(len(dev_loader.dataset))
print(len(test_loader.dataset))

8027
993
980


In [19]:
for item in train_loader:
    # print(item)
    # audio features, 40x100 , 
    #40 is the number of features,
    #100 is the number of frames, 1024 is the batch size
    print(item[0].shape)
    # keyword, 1024 is the batch size 
    print(item[1].shape)
    exm_input = item[0]
    break

torch.Size([1024, 40, 100])
torch.Size([1024])


In [28]:
## evaluate metrics
import evaluate

metric = evaluate.load("wer")

In [35]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [29]:
# load a pre-trained checkpoint
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")


In [30]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [33]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-en",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=4000,
    gradient_checkpointing=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

In [36]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_loader.dataset,
    eval_dataset=test_loader.dataset,
    compute_metrics=compute_metrics,
)

In [38]:
# trainer.train()

